In [53]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
#from keras.models import load_model
import urllib.request
from urllib3 import PoolManager
import re

In [52]:
#Gjenstår: Hent odds

## Find odds for 7 days at a time.

### Source 1: soccerbase

In [3]:
import gzip
import urllib.request
games = []

In [3]:

# url_string2 = None
# with urllib.request.urlopen('http://www.soccerbase.com/matches/home.sd?type=2') as response:
#     gzipFile = gzip.GzipFile(fileobj=response)
#     url_string2 = gzipFile.read().decode("UTF-8")
#     url_string2 = re.search('(?<=soccer.gamelist.Timekeeper.addGames\({).*?(?=soccer.gamelist)', url_string2)[0]


In [4]:
# matches = re.finditer('(?<=\=)(.*?)\_V\_(.*?)\=(.*?)\"', url_string2)
# for match in matches:
#     games.append([match[3],re.sub("_", " ", match[1]).lower(),re.sub("_", " ", match[2]).lower()])

### Source 2: supersport

In [5]:
# response = urllib.request.urlopen('https://www.supersport.com/football/fixtures')
# url_string = response.read()
# url_string = url_string.decode("UTF-8")
# url_string = re.search('(?<=FIXTURES)(.|\s)*', url_string)[0]

In [6]:
# dates = re.finditer("(?<=Header\">)(.*?)(?=class=\"table)", url_string)

# for day in dates:
    
#     date = re.search('.*?(?=</td>)',day[0])[0]
#     matches = re.finditer("(?<=>)(..:..)(.*?)(?=</td><td class)",day[0])
    
#     for match in matches:
#         time_of_match = match[0][:5]
#         teams_re = re.finditer("(?<=st\'>)(.*?)(?=</td>)", match[0])
#         game = [date]
#         for team in teams_re:
            
#             game.append(team[0].lower())
#         games.append(game)

### Source 3: thefishy

In [4]:
url_string3 = None
with urllib.request.urlopen('https://thefishy.co.uk/nextweeksfixtures.php') as response:
    url_string3 = response.read().decode("UTF-8") + " 12/12</th>"

In [5]:
dates = re.finditer("(\ ../..<(.|\s)*?)(?=\ \w\w/\w\w</th>)", url_string3)

for date in dates:
    day = date[0][1:6] + "/18"
    matches = re.finditer("(?<=\'>)(.*?)\ v\ (.*?)(?=<)", date[0])
    for match in matches:
        away_team = match[2]
        home_team = re.search('(?<=team2)(.*?>)(.*?)(?=$)',match[1])[2]
        
#         print(home_team, away_team)
        
        games.append([day, home_team.lower(), away_team.lower()])


In [6]:
%run update_stash.py

unzipping  ../../Football_Data/Zipstash\data_0.zip
unzipping  ../../Football_Data/Zipstash\data_1.zip
unzipping  ../../Football_Data/Zipstash\data_10.zip
unzipping  ../../Football_Data/Zipstash\data_11.zip
unzipping  ../../Football_Data/Zipstash\data_12.zip
unzipping  ../../Football_Data/Zipstash\data_13.zip
unzipping  ../../Football_Data/Zipstash\data_14.zip
unzipping  ../../Football_Data/Zipstash\data_15.zip
unzipping  ../../Football_Data/Zipstash\data_16.zip
unzipping  ../../Football_Data/Zipstash\data_17.zip
unzipping  ../../Football_Data/Zipstash\data_18.zip
unzipping  ../../Football_Data/Zipstash\data_19.zip
unzipping  ../../Football_Data/Zipstash\data_2.zip
unzipping  ../../Football_Data/Zipstash\data_20.zip
unzipping  ../../Football_Data/Zipstash\data_21.zip
unzipping  ../../Football_Data/Zipstash\data_22.zip
unzipping  ../../Football_Data/Zipstash\data_23.zip
unzipping  ../../Football_Data/Zipstash\data_24.zip
unzipping  ../../Football_Data/Zipstash\data_25.zip
unzipping  ../.

In [27]:
all_seasons = []
for filename in os.listdir('C:/Users/bfesc/Documents/Football_Data/Stash/'):
    #if filename[0] == 'B':
    with open("C:/Users/bfesc/Documents/Football_Data/Stash/" + filename) as file:
        season = file.read().rstrip().split('\n')
        for i in range(len(season)):
            season[i] = season[i].rstrip(',').split(',')
            for j in range(len(season[i])):
                season[i][j] = season[i][j].lower()
        all_seasons.append(season)

In [40]:
columns = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'WHH', 'WHD', 'WHA']
columns = [col.lower() for col in columns]
all_data = pd.DataFrame(columns=columns)
for season in all_seasons:
    if not np.in1d(columns, season[0]).all():
        continue
    if len(season[0]) != max([len(s) for s in season]):
        for i in range(len(season)):
            season[i] = season[i][:len(season[0])]
    df = pd.DataFrame(season[1:])
    df.columns = season[0]
    all_data = all_data.append(df[columns])
    
all_data['date'] = pd.to_datetime(all_data['date'], dayfirst=True)
all_data.index = all_data['date']
all_data = all_data.drop(columns=['date','div'])
for column in all_data.columns:
    all_data = all_data[pd.notnull(all_data[column])]
all_data = all_data[pd.notnull(all_data.index)]
all_data = all_data.iloc[np.where(all_data['whh'] != "")]
all_data = all_data.iloc[np.where(all_data['whd'] != "")]
all_data = all_data.iloc[np.where(all_data['wha'] != "")]
all_data = all_data.iloc[np.where(all_data['fthg'] != "")]
all_data = all_data.sort_index(ascending=False)
all_data['ftr'].iloc[np.where(all_data['ftr'] == 'a')] = 0
all_data['ftr'].iloc[np.where(all_data['ftr'] == 'd')] = 1
all_data['ftr'].iloc[np.where(all_data['ftr'] == 'h')] = 2
all_data['fthg'] = all_data['fthg'].astype(int)
all_data['ftag'] = all_data['ftag'].astype(int)
all_data['whh'] = all_data['whh'].astype(np.float32)
all_data['whd'] = all_data['whd'].astype(np.float32)
all_data['wha'] = all_data['wha'].astype(np.float32)

In [12]:
from difflib import SequenceMatcher

def unknown_team(home_team, away_team, team_dict, unknown = "away"):

    if unknown == "away":
        team_scores = []
        teams = team_dict[home_team]['awayteam'] #Away or HomeTeam doesnt really matter 
        
        for team in teams:
            team_scores.append(SequenceMatcher(None,team,away_team).ratio())
        
        team_scores = np.argsort(team_scores)
        
        away_team = teams[team_scores[-1]]
    
    else:
        teams = team_dict[away_team]['hometeam']
        team_scores = []
            
        for team in teams:
            team_scores.append(SequenceMatcher(None,team,home_team).ratio())
        
        team_scores = np.argsort(team_scores)
        
        home_team = teams[team_scores[-1]]
        
        #if team_scores[-1] < 0.4
        
    return home_team, away_team


In [23]:
"""def make_team_dicts(data):
    all_teams = np.unique(np.append(data['HomeTeam'].str.lower(), data['AwayTeam'].str.lower()))
    team_dict = {}
    
    for team in all_teams:
        team_dict[team] = data.iloc[np.where(np.logical_or(data['HomeTeam'] == team, 
                                                      data['AwayTeam'] == team))].sort_index(ascending=False)
        
    return team_dict"""

In [42]:
def make_team_dicts(data):
    
    ht = data["hometeam"].str.lower()
    at = data["awayteam"].str.lower()
    
    all_teams = np.unique(np.append(ht, at))
    team_dict = {}
    
    for team in all_teams:
        team_dict[team] = data.iloc[np.where(np.logical_or(ht == team, 
                                                      at == team))].sort_index(ascending=False)
        
    return team_dict

In [43]:
team_dict = make_team_dicts(all_data)

In [44]:
# Sanity check
unknown_team("dortmund", "bayern münchen", team_dict)

('dortmund', 'bayern munich')

In [51]:
teststr = """":{"topic":"PDS/OB_EV20106699/OB_MA934625950","hash":[true,"A","",false,null,null,null,false,true,false,"MR",null,null],"parent":"OB_EV20106699"},"OB_MA939935498":{"topic":"PDS/OB_EV20240488/OB_MA939935498","hash":[true,"A","",false,null,null,null,false,true,false,"MR",null,null],"parent":"OB_EV20240488"},"OB_MA939458334":{"topic":"PDS/OB_EV20228720/OB_MA939458334","hash":[true,"A","",false,null,null,null,false,true,false,"MR",null,null],"parent":"OB_EV20228720"},"OB_MA934626127":{"topic":"PDS/OB_EV20106699/OB_MA934626127","hash":[true,"A","",false,null,null,null,false,true,false,"--",null,null],"parent":"OB_EV20106699"},"OB_MA934625949":{"topic":"PDS/OB_EV20106699/OB_MA934625949","hash":[true,"A","",false,null,null,null,false,true,false,"--",null,null],"parent":"OB_EV20106699"},"OB_MA934625960":{"topic":"PDS/OB_EV20106699/OB_MA934625960","hash":[true,"A","",false,null,null,null,false,true,false,"MR",null,null],"parent":"OB_EV20106699"},"OB_MA937690509":{"topic":"PDS/OB_EV20185837/OB_MA937690509","hash":[true,"A","",false,null,null,null,false,true,false,"--",null,null],"parent":"OB_EV20185837"},"OB_MA939935608":{"topic":"PDS/OB_EV20240488/OB_MA939935608","hash":[true,"A","",false,null,null,null,false,true,false,"--",null,null],"parent":"OB_EV20240488"},"OB_MA934625864":{"topic":"PDS/OB_EV20106699/OB_MA934625864","hash":[true,"A","",false,null,null,null,false,true,false,"DC",null,null],"parent":"OB_EV20106699"}},"selections":{"OB_OU3121467567":{"topic":"PDS/OB_EV20185837/OB_MA937690511/OB_OU3121467567","hash":[true,"A","3","10",null,null,null,null,false,"H"],"parent":"OB_MA937690511"},"OB_OU3110580908":{"topic":"PDS/OB_EV20106699/OB_MA934625950/OB_OU3110580908","hash":[true,"A","23","20",null,null,null,null,false,"A"],"parent":"OB_MA934625950"},"OB_OU3129604949":{"topic":"PDS/OB_EV20240488/OB_MA939935498/OB_OU3129604949","hash":[true,"A","11","20",null,null,null,null,false,"H"],"parent":"OB_MA939935498"},"OB_OU3127895498":{"topic":"PDS/OB_EV20228720/OB_MA939458334/OB_OU3127895498","hash":[true,"A","13","20",null,null,null,null,false,"H"],"parent":"OB_MA939458334"},"OB_OU3110581538":{"topic":"PDS/OB_EV20106699/OB_MA934626127/OB_OU3110581538","hash":[true,"A","1","1",null,null,null,null,false,"-"],"parent":"OB_MA934626127"},"OB_OU3110580910":{"topic":"PDS/OB_EV20106699/OB_MA934625949/OB_OU3110580910","hash":[true,"A","2","5",null,null,null,null,false,"-"],"parent":"OB_MA934625949"},"OB_OU3110580919":{"topic":"PDS/OB_EV20106699/OB_MA934625960/OB_OU3110580919","hash":[true,"A","15","4",null,null,null,null,false,"A"],"parent":"OB_MA934625960"},"OB_OU3121467580":{"topic":"PDS/OB_EV20185837/OB_MA937690509/OB_OU3121467580","hash":[true,"A","1","5",null,null,null,null,false,"-"],"parent":"OB_MA937690509"},"OB_OU3129605353":{"topic":"PDS/OB_EV20240488/OB_MA939935608/OB_OU3129605353","hash":[true,"A","2","5",null,null,null,null,false,"-"],"parent":"OB_MA939935608"},"OB_OU3110580641":{"topic":"PDS/OB_EV20106699/OB_MA934625864/OB_OU3110580641","hash":[true,"A","2","7",null,null,null,null,false,"2"],"parent":"OB_MA934625864"}}};</script><div class="topbets clickable-selections" data-panel="panel-top-bets" aria-label="Top Bets"><section class="header-panel"><div class="header-panel__content"><div class="header-panel__content-inner"><span class="topbets__header-icon icon-most-popular-bets2"></span><h2 class="header-panel__title -text-left">Top Bets</h2></div></div></section><div class="topbets__header"><h2>Top Bets</h2><hr class="topbets__header--separator"/></div><ul class="grid grid--wrap grid--aitems-center"><li class="topbets__list-item" id="OB_EV20185837" data-entityid="OB_EV20185837" data-displayed="true" data-betinrun="false"><div class="grid grid--aitems-center"><div class="grid-col"><div><i class="topbets__icon icon-football"></i> <span class="topbets__icon-name">Football |</span> <time data-type="event-starttime" datetime="2021-04-11T19:00:00.000Z" class="eventStartTime localisable topbets__list-item--time"></time></div><h3 class="topbets__list-item--event">Lyon - 90 Minutes</h3><a class="topbets__list-item--link" href="/betting/en-gb/football/OB_EV20185837/lyon-vs-angers">Lyon v Angers</a></div><div id="OB_MA937690511" class="grid-col grid-col--shrink" data-entityid="OB_MA937690511" data-displayed="true" data-status="A" data-betinrun="false"><button id="OB_OU3121467567" class="btn betbutton oddsbutton" aria-label="Lyon at 3/10" data-name="Lyon" data-displayed="true" data-status="A" data-player="Lyon" data-entityid="OB_OU3121467567" data-market="OB_MA937690511" data-event="OB_EV20185837" data-odds="3/10" data-num="3" data-denom="10" data-goal-name="Goal"><span class="betbutton__odds">3/10</span> <span class="icon"></span></button></div></div></li><li class="topbets__list-item" id="OB_EV20106699" data-entityid="OB_EV20106699" data-displayed="true" data-betinrun="false"><div class="grid grid--aitems-center"><div class="grid-col"><div><i class="topbets__icon icon-football"></i> <span class="topbets__icon-name">Football |</span> <time data-type="event-starttime" datetime="2021-04-11T19:00:00.000Z" class="eventStartTime localisable topbets__list-item--time"></time></div><h3 class="topbets__list-item--event">Atletico Madrid - 90 Minutes</h3><a class="topbets__list-item--link" href="/betting/en-gb/football/OB_EV20106699/real-betis-vs-atletico-madrid">Real Betis v Atletico Madrid</a></div><div id="OB_MA934625950" class="grid-col grid-col--shrink" data-entityid="OB_MA934625950" data-displayed="true" data-status="A" data-betinrun="false"><button id="OB_OU3110580908" class="btn betbutton oddsbutton" aria-label="Atletico Madrid at 23/20" data-name="Atletico Madrid" data-displayed="true" data-status="A" data-player="Atletico Madrid" data-entityid="OB_OU3110580908" data-market="OB_MA934625950" data-event="OB_EV20106699" data-odds="23/20" data-num="23" data-denom="20" data-goal-name="Goal"><span class="betbutton__odds">23/20</span> <span class="icon"></span></button></div></div></li><li class="topbets__list-item" id="OB_EV20240488" data-entityid="OB_EV20240488" data-displayed="true" data-betinrun="false"><div class="grid grid--aitems-center"><div class="grid-col"><div><i class="topbets__icon icon-football"></i> <span class="topbets__icon-name">Football |</span> <time data-type="event-starttime" datetime="2021-04-11T19:00:00.000Z" class="eventStartTime localisable topbets__list-item--time"></time></div><h3 class="topbets__list-item--event">Sporting Lisbon - 90 Minutes</h3><a class="topbets__list-item--link" href="/betting/en-gb/football/OB_EV20240488/sporting-lisbon-vs-famalicao">Sporting Lisbon v Famalicao</a></div><div id="OB_MA939935498" class="grid-col grid-col--shrink" data-entityid="OB_MA939935498" data-displayed="true" data-status="A" data-betinrun="false"><button id="OB_OU3129604949" class="btn betbutton oddsbutton" aria-label="Sporting Lisbon at 11/20" data-name="Sporting Lisbon" data-displayed="true" data-status="A" data-player="Sporting Lisbon" data-entityid="OB_OU3129604949" data-market="OB_MA939935498" data-event="OB_EV20240488" data-odds="11/20" data-num="11" data-denom="20" data-goal-name="Goal"><span class="betbutton__odds">11/20</span> <span class="icon"></span></button></div></div></li><li class="topbets__list-item" id="OB_EV20228720" data-entityid="OB_EV20228720" data-displayed="true" data-betinrun="false"><div class="grid grid--aitems-center"><div class="grid-col"><div><i class="topbets__icon icon-football"></i> <span class="topbets__icon-name">Football |</span> <time data-type="event-starttime" datetime="2021-04-11T19:00:00.000Z" class="eventStartTime localisable topbets__list-item--time"></time></div><h3 class="topbets__list-item--event">Espanyol - 90 Minutes</h3><a class="topbets__list-item--link" href="/betting/en-gb/football/OB_EV20228720/espanyol-vs-leganes">Espanyol v Leganes</a></div><div id="OB_MA939458334" class="grid-col grid-col--shrink" data-entityid="OB_MA939458334" data-displayed="true" data-status="A" data-betinrun="false"><button id="OB_OU3127895498" class="btn betbutton oddsbutton" aria-label="Espanyol at 13/20" data-name="Espanyol" data-displayed="true" data-status="A" data-player="Espanyol" data-entityid="OB_OU3127895498" data-market="OB_MA939458334" data-event="OB_EV20228720" data-odds="13/20" data-num="13" data-denom="20" data-goal-name="Goal"><span class="betbutton__odds">13/20</span> <span class="icon"></span></button></div></div></li><li class="topbets__list-item" id="OB_EV20106699" data-entityid="OB_EV20106699" data-displayed="true" data-betinrun="false"><div class="grid grid--aitems-center"><div class="grid-col"><div><i class="topbets__icon icon-football"></i> <span class="topbets__icon-name">Football |</span> <time data-type="event-starttime" datetime="2021-04-11T19:00:00.000Z" class="eventStartTime localisable topbets__list-item--time"></time></div><h3 class="topbets__list-item--event">Yes - Both Teams To Score</h3><a class="topbets__list-item--link" href="/betting/en-gb/football/OB_EV20106699/real-betis-vs-atletico-madrid">Real Betis v Atletico Madrid</a></div><div id="OB_MA934626127" class="grid-col grid-col--shrink" data-entityid="OB_MA934626127" data-displayed="true" data-status="A" data-betinrun="false"><button id="OB_OU3110581538" class="btn betbutton oddsbutton" aria-label="Yes at 1/1" data-name="Yes" data-displayed="true" data-status="A" data-player="Yes" data-entityid="OB_OU3110581538" data-market="OB_MA934626127" data-event="OB_EV20106699" data-odds="1/1" data-num="1" data-denom="1" data-goal-name="Goal"><span class="betbutton__odds">1/1</span> <span class="icon"></span></button></div></div></li><li class="topbets__list-item" id="OB_EV20106699" data-entityid="OB_EV20106699" data-displayed="true" data-betinrun="false"><div class="grid grid--aitems-center"><div class="grid-col"><div><i class="topbets__icon icon-football"></i> <span class="topbets__icon-name">Football |</span> <time data-type="event-starttime" datetime="2021-04-11T19:00:00.000Z" class="eventStartTime localisable topbets__list-item--time"></time></div><h3 class="topbets__list-item--event">Over 1.5 - Total Match Goals Over/Under 1.5 Goals</h3><a class="topbets__list-item--link" href="/betting/en-gb/football/OB_EV20106699/real-betis-vs-atletico-madrid">Real Betis v Atletico Madrid</a></div><div id="OB_MA934625949" class="grid-col grid-col--shrink" data-entityid="OB_MA934625949" data-displayed="true" data-status="A" data-betinrun="false"><button id="OB_OU3110580910" class="btn betbutton oddsbutton" aria-label="Over 1.5 at 2/5" data-name="Over 1.5" data-displayed="true" data-status="A" data-player="Over 1.5" data-entityid="OB_OU3110580910" data-market="OB_MA934625949" data-event="OB_EV20106699" data-odds="2/5" data-num="2" data-denom="5" data-goal-name="Goal"><span class="betbutton__odds">2/5</span> <span class="icon"></span></button></div></div></li><li class="topbets__list-item" id="OB_EV20106699" data-entityid="OB_EV20106699" data-displayed="true" data-betinrun="false"><div class="grid grid--aitems-center"><div class="grid-col"><div><i class="topbets__icon icon-football"></i> <span class="topbets__icon-name">Football |</span> <time data-type="event-starttime" datetime="2021-04-11T19:00:00.000Z" class="eventStartTime localisable topbets__list-item--time"></time></div><h3 class="topbets__list-item--event">Atletico Madrid - Match Result and Both Teams To Score</h3><a class="topbets__list-item--link" href="/betting/en-gb/football/OB_EV20106699/real-betis-vs-atletico-madrid">Real Betis v Atletico Madrid</a></div><div id="OB_MA934625960" class="grid-col grid-col--shrink" data-entityid="OB_MA934625960" data-displayed="true" data-status="A" data-betinrun="false"><button id="OB_OU3110580919" class="btn betbutton oddsbutton" aria-label="Atletico Madrid at 15/4" data-name="Atletico Madrid" data-displayed="true" data-status="A" data-player="Atletico Madrid" data-entityid="OB_OU3110580919" data-market="OB_MA934625960" data-event="OB_EV20106699" data-odds="15/4" data-num="15" data-denom="4" data-goal-name="Goal"><span class="betbutton__odds">15/4</span> <span class="icon"></span></button></div></div></li><li class="topbets__list-item" id="OB_EV20185837" data-entityid="OB_EV20185837" data-displayed="true" data-betinrun="false"><div class="grid grid--aitems-center"><div class="grid-col"><div><i class="topbets__icon icon-football"></i> <span class="topbets__icon-name">Football |</span> <time data-type="event-starttime" datetime="2021-04-11T19:00:00.000Z" class="eventStartTime localisable topbets__list-item--time"></time></div><h3 class="topbets__list-item--event">Over 1.5 - Total Match Goals Over/Under 1.5 Goals</h3><a class="topbets__list-item--link" href="/betting/en-gb/football/OB_EV20185837/lyon-vs-angers">Lyon v Angers</a></div><div id="OB_MA937690509" class="grid-col grid-col--shrink" data-entityid="OB_MA937690509" data-displayed="true" data-status="A" data-betinrun="false"><button id="OB_OU3121467580" class="btn betbutton oddsbutton" aria-label="Over 1.5 at 1/5" data-name="Over 1.5" data-displayed="true" data-status="A" data-player="Over 1.5" data-entityid="OB_OU3121467580" data-market="OB_MA937690509" data-event="OB_EV20185837" data-odds="1/5" data-num="1" data-denom="5" data-goal-name="Goal"><span class="betbutton__odds">1/5</span> <span class="icon"></span></button></div></div></li><li class="topbets__list-item" id="OB_EV20240488" data-entityid="OB_EV20240488" data-displayed="true" data-betinrun="false"><div class="grid grid--aitems-center"><div class="grid-col"><div><i class="topbets__icon icon-football"></i> <span class="topbets__icon-name">Football |</span> <time data-type="event-starttime" datetime="2021-04-11T19:00:00.000Z" class="eventStartTime localisable topbets__list-item--time"></time></div><h3 class="topbets__list-item--event">Over 1.5 - Total Match Goals Over/Under 1.5 Goals</h3><a class="topbets__list-item--link" href="/betting/en-gb/football/OB_EV20240488/sporting-lisbon-vs-famalicao">Sporting Lisbon v Famalicao</a></div><div id="OB_MA939935608" class="grid-col grid-col--shrink" data-entityid="OB_MA939935608" data-displayed="true" data-status="A" data-betinrun="false"><button id="OB_OU3129605353" class="btn betbutton oddsbutton" aria-label="Over 1.5 at 2/5" data-name="Over 1.5" data-displayed="true" data-status="A" data-player="Over 1.5" data-entityid="OB_OU3129605353" data-market="OB_MA939935608" data-event="OB_EV20240488" data-odds="2/5" data-num="2" data-denom="5" data-goal-name="Goal"><span class="betbutton__odds">2/5</span> <span class="icon"></span></button></div></div></li><li class="topbets__list-item" id="OB_EV20106699" data-entityid="OB_EV20106699" data-displayed="true" data-betinrun="false"><div class="grid grid--aitems-center"><div class="grid-col"><div><i class="topbets__icon icon-football"></i> <span class="topbets__icon-name">Football |</span> <time data-type="event-starttime" datetime="2021-04-11T19:00:00.000Z" class="eventStartTime localisable topbets__list-item--time"></time></div><h3 class="topbets__list-item--event">Atletico Madrid ... - Double Chance</h3><a class="topbets__list-item--link" href="/betting/en-gb/football/OB_EV20106699/real-betis-vs-atletico-madrid">Real Betis v Atletico Madrid</a></div><div id="OB_MA934625864" class="grid-col grid-col--shrink" data-entityid="OB_MA934625864" data-displayed="true" data-status="A" data-betinrun="false"><button id="OB_OU3110580641" class="btn betbutton oddsbutton" aria-label="Atletico Madrid ... at 2/7" data-name="Atletico Madrid ..." data-displayed="true" data-status="A" data-player="Atletico Madrid ..." data-entityid="OB_OU3110580641" data-market="OB_MA934625864" data-event="OB_EV20106699" data-odds="2/7" data-num="2" data-denom="7" data-goal-name="Goal"><span class="betbutton__odds">2/7</span> <span class="icon"></span></button></div></div></li></ul><div class="topbets__footer -text-center"><a href="/betting/en-gb/apps/top-bets">View All Top Bets</a></div></div></div></div></aside></div><div id="betslip-overlay" class="wh-overlay wh-overlay--blue-dark betslip-overlay"></div></div><style>.sb-scroll-to-top{"""
re.findall(".........................................Lyon v Angers.......................................",teststr)

['b/football/OB_EV20185837/lyon-vs-angers">Lyon v Angers</a></div><div id="OB_MA937690511" clas',
 'b/football/OB_EV20185837/lyon-vs-angers">Lyon v Angers</a></div><div id="OB_MA937690509" clas']

In [162]:
def make_live_data(live_data, team_dict):


    training_data = pd.DataFrame(columns=['Date','HomeTeam', 'AwayTeam', 'DaysSinceLastHTMatch', 'DaysSinceLastATMatch', 'HomeTeamForm', 'AwayTeamForm',
                                          'HomeTeamHomeForm','AwayTeamAwayForm', 'HomeTeamOffense', 'HomeTeamDefense',
                                          'AwayTeamOffense', 'AwayTeamDefense', 'LastInterTeamGame', 
                                           'Result'])


    history = pd.Timedelta('90d')
    year = pd.Timedelta('365d')
    for i in tqdm(range(len(live_data))):#
        game = live_data[i]
        home_team = game[1]
        away_team = game[2]
        game_date = pd.to_datetime(game[0]) 

        if home_team not in team_dict.keys() and away_team not in team_dict.keys():
            continue
        elif home_team not in team_dict.keys():
            home_team, away_team = unknown_team(home_team, away_team, team_dict, unknown="home")
        elif away_team not in team_dict.keys():
            home_team, away_team = unknown_team(home_team, away_team, team_dict, unknown="away")
        
        ht_games = team_dict[home_team].loc[game_date - pd.Timedelta('1h'):game_date - year]
        at_games = team_dict[away_team].loc[game_date - pd.Timedelta('1h'):game_date - year]

        #result = game['FTR']
        result = 0

        ht_hgames = ht_games.iloc[np.where(ht_games['hometeam'] == home_team)]
        ht_agames = ht_games.iloc[np.where(ht_games['awayteam'] == home_team)]

        at_hgames = at_games.iloc[np.where(at_games['hometeam'] == away_team)]
        at_agames = at_games.iloc[np.where(at_games['awayteam'] == away_team)]


        if len(ht_hgames) > 1 and len(at_agames) > 1:

            ht_time_diff = np.log((game_date - ht_games.index).days.values + 1)
            at_time_diff = np.log((game_date - at_games.index).days.values + 1)

            ht_h_td = ht_time_diff[np.where(ht_games['hometeam'] == home_team)]
            ht_a_td = ht_time_diff[np.where(ht_games['awayteam'] == home_team)]
            at_h_td = at_time_diff[np.where(at_games['hometeam'] == away_team)]
            at_a_td = at_time_diff[np.where(at_games['awayteam'] == away_team)]


            it_games = np.append(np.where(ht_hgames['awayteam'] == away_team)[0],
                                 np.where(ht_agames['hometeam'] == away_team)[0])

            litgr = 0
            if len(it_games) > 0:
                litgr += np.mean((ht_games['fthg'].iloc[it_games].values - ht_games['ftag'].iloc[it_games].values) / ht_time_diff[it_games])

            dslhtm = ht_time_diff[0]
            dslatm = at_time_diff[0]

            ht_form = np.mean(np.append(((ht_hgames['fthg'].values - ht_hgames['ftag'].values) / ht_h_td),
                              (ht_agames['ftag'].values - ht_agames['fthg'].values) / ht_a_td))
            at_form = np.mean(np.append((at_hgames['fthg'].values - at_hgames['ftag'].values) / at_h_td,
                              (at_agames['ftag'].values - at_agames['fthg'].values) / at_a_td))

            ht_hform = np.mean((ht_hgames['fthg'].values - ht_hgames['ftag'].values) / ht_h_td)
            at_aform = np.mean((at_agames['ftag'].values - at_agames['fthg'].values) / at_a_td)

            hto = np.mean(np.append(ht_hgames['fthg'].values / ht_h_td, ht_agames['ftag'].values / ht_a_td))
            htd = -np.mean(np.append(ht_hgames['ftag'].values / ht_h_td, ht_agames['fthg'].values /ht_a_td))

            ato = np.mean(np.append(at_hgames['fthg'] / at_h_td, at_agames['ftag'].values / at_a_td))
            atd = -np.mean(np.append(at_hgames['ftag'].values / at_h_td, at_agames['fthg'].values / at_a_td))

            
            training_data = training_data.append(pd.DataFrame([[game_date,home_team, away_team, dslhtm, dslatm, ht_form, at_form, ht_hform, at_aform,
                                                                hto, htd, ato, atd, litgr, result]], columns=training_data.columns))

    return training_data
    
    

In [163]:
training_data = make_live_data(games,team_dict)

100%|███████████████████████████████████████████████████████████████████████████████| 686/686 [00:05<00:00, 126.56it/s]


In [164]:
training_data = training_data.reset_index(drop=True)
training_data.mean()

DaysSinceLastHTMatch    1.203411
DaysSinceLastATMatch    1.186577
HomeTeamForm           -0.002533
AwayTeamForm            0.002356
HomeTeamHomeForm        0.053048
AwayTeamAwayForm       -0.067949
HomeTeamOffense         0.304641
HomeTeamDefense        -0.307175
AwayTeamOffense         0.317741
AwayTeamDefense        -0.315385
LastInterTeamGame       0.031931
Result                  0.000000
dtype: float64

In [165]:
training_data.shape

(206, 15)

In [166]:
teams_playing = np.unique(np.array(games)[:,1].tolist() + np.array(games)[:,2].tolist())

In [167]:
def get_odds_betfair():
    html = PoolManager()
    webpage = html.request("GET","https://www.betfair.com/sport/football")

    webpage_l = webpage.data.decode("UTF-8").lower()

    end_cue = "</span>\n</div>\n</a>\n</div>\n</div>\n</div>\n</li>"
    start_cue = '<div class="avb-col avb-col-markets">'

    wp_all_games = re.findall(f"{start_cue}.*?{end_cue}",webpage_l,flags=re.DOTALL)

    game_odds = []

    for wp in wp_all_games:
        try:
            odds = [float(o) for o in re.findall("(?<=\n)\d{1,2}\.\d{1,2}(?=\n)",wp)[2:]]
            hometeam, awayteam = re.findall('<span class="team-name" title="(.*?)">',wp)

            game_odds.append({"hometeam" : hometeam,
                                "awayteam" : awayteam,
                                "homeodds" : odds[0],
                                "awayodds" : odds[-1]})
        except Exception as e:
            print(e)

    return game_odds

In [168]:
odds = pd.DataFrame(get_odds_betfair())

list index out of range
list index out of range


In [169]:
odds = odds.loc[~((odds.homeodds > 3) & (odds.awayodds > 3))]

In [188]:
#inntil videre så settes treff på minst ett lag som krav.
print(odds.loc[odds.hometeam.isin(teams_playing) | odds.awayteam.isin(teams_playing)].shape)
data_with_odds = training_data.merge(odds,left_on="HomeTeam",right_on="hometeam",how="inner")
data_with_odds = data_with_odds.append(training_data.merge(odds,left_on="AwayTeam",right_on="awayteam",how="inner"))
data_with_odds.shape[0]

(42, 4)


15

In [194]:
from xgboost import XGBClassifier
gb = XGBClassifier()
gb.load_model("model_2021.json")

In [208]:
pred_cols = ['DaysSinceLastHTMatch', 'DaysSinceLastATMatch', 'HomeTeamForm', 'AwayTeamForm',
      'HomeTeamHomeForm','AwayTeamAwayForm', 'HomeTeamOffense', 'HomeTeamDefense',
      'AwayTeamOffense', 'AwayTeamDefense', 'LastInterTeamGame', 
      'HomeTeamOdds', 'AwayTeamOdds']

livepreds = gb.predict(data_with_odds.rename(columns={"homeodds":"HomeTeamOdds","awayodds":"AwayTeamOdds"})[pred_cols])

In [214]:
def bet_size(cash, n_bets):
    return (cash*0.25) / n_bets

print(f"bet_size = {bet_size(1000,len(livepreds))}")
for i in range(len(livepreds)):
    print(f"{data_with_odds.iloc[i].Date}: {data_with_odds.iloc[i].HomeTeam} - {data_with_odds.iloc[i].AwayTeam} ")
    print(["Away","Draw","Home"][livepreds[i]])
    print()

bet_size = 16.666666666666668
2018-11-04 00:00:00: west brom - southampton 
Away

2018-11-04 00:00:00: brighton - everton 
Home

2018-11-04 00:00:00: brighton - man city 
Home

2018-11-04 00:00:00: hoffenheim - leverkusen 
Away

2018-11-04 00:00:00: benevento - sassuolo 
Away

2018-11-04 00:00:00: guingamp - toulouse 
Away

2018-11-04 00:00:00: kasimpasa - yeni malatyaspor 
Home

2018-11-04 00:00:00: west brom - southampton 
Away

2018-11-04 00:00:00: brighton - everton 
Home

2018-11-04 00:00:00: celta - sevilla 
Away

2018-11-04 00:00:00: sociedad - sevilla 
Away

2018-11-04 00:00:00: hoffenheim - leverkusen 
Away

2018-11-04 00:00:00: benevento - sassuolo 
Away

2018-11-04 00:00:00: guingamp - toulouse 
Away

2018-11-04 00:00:00: giannina - lamia 
Home



In [207]:
livepreds[i]

array([0.46973372, 0.269813  , 0.26045328], dtype=float32)